In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import itertools
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn import metrics

from sqlalchemy import create_engine
import datetime 
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib
pd.options.display.max_columns=300
pd.options.display.max_rows=100
# from TurbineTimeSeries.storage import MachineDataStore
plt.rcParams["figure.figsize"] = (14,8)
#from TurbineTimeSeries.transformations import PCA, StandardScaler, DropCols, DropSparseCols, LeftJoin
%matplotlib inline

In [2]:
# model_number = 2

# store = MachineDataStore('.config')

# # model_data_hr = (store.query(model_number,'1hr')
# #                  .not_null(['timestamp','psn'])
# # #                  .exclude_psn([44,52,54,70])
# #                  .execute())

# model_data_min = (store.query(model_number,'10min')
#                   .not_null(['timestamp','psn'])
#                   .exclude_psn([44,52,54,70])
#                   .execute())

# model_data_min.head()
# model1_1hr = pd.read_csv('../../../data/raw_data_model1.csv',index_col=0)
# model2_1hr = pd.read_csv('../../../data/raw_data_model2.csv',index_col=0)

# model1_10min = pd.read_csv('../../../data/raw_data_model1_10min.csv',index_col=0)
model2_10min = pd.read_csv('../../../data/raw_data_model2_10min.csv')#,index_col=0)

## convert all to timestamps
# model1_1hr['timestamp'] = model1_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model2_1hr['timestamp'] = model2_1hr['timestamp'].apply(lambda x: pd.Timestamp(x))
# model1_10min['timestamp'] = model1_10min['timestamp'].apply(lambda x: pd.Timestamp(x))
model2_10min['timestamp'] = model2_10min['timestamp'].apply(lambda x: pd.Timestamp(x))


model2_10min = model2_10min.sort_values(by=['psn','timestamp'],ascending=[True,True])

# print('Shape of model1 1hr data: ', model1_1hr.shape)
# print('Shape of model2 1hr data: ', model2_1hr.shape)
# print('Shape of model1 10min data: ', model1_10min.shape)
print('Shape of model2 10min data: ', model2_10min.shape)

Shape of model2 10min data:  (1602326, 76)


In [37]:
def shutdown_labeler(df,eng_st_col= 'sum_eng_st'):
    shutdown_df = pd.DataFrame(index=model2_10min.index.values, columns = model2_10min.columns.values)
    shutdown_df['shutdown_flag']=0
    for psn in model2_10min['psn'].unique():
        tempdf = model2_10min[model2_10min['psn']==psn]
        shutdown_flag = np.diff(tempdf[eng_st_col])
        tempdf['shutdown_flag'] = np.array(np.append(shutdown_flag,[0]),dtype=bool) ## cast everything to boolean
        shutdown_df = shutdown_df.append(tempdf)
    return(shutdown_df)

In [38]:
model2_10min_with_shutdowns = shutdown_labeler(model2_10min)

MemoryError: 

In [31]:
shutdown_df = pd.DataFrame(index=model2_10min.index.values, columns = model2_10min.columns.values)
shutdown_df['shutdown_flag']=0
for psn in model2_10min['psn'].unique():
    tempdf = model2_10min[model2_10min['psn']==psn]
    shutdown_flag = np.diff(tempdf['sum_eng_st'])
    tempdf['shutdown_flag'] = np.array(np.append(shutdown_flag,[0]),dtype=bool) ## cast everything to boolean
    shutdown_df = shutdown_df.append(tempdf)

C:\Users\d59628\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


MemoryError: 

In [34]:
np.array(np.array([1,2,3,4,0,1]),dtype=bool)

array([ True,  True,  True,  True, False,  True])

In [24]:
for i,j in enumerate(np.diff(model2_10min[model2_10min['psn']==46]['sum_eng_st'])):
    if j > 0:
        print(i,j)

1205 1.0
3183 1.0
3353 1.0
3538 4.0
3591 1.0
6349 1.0


In [25]:
testdf = model2_10min[model2_10min['psn']==46]

In [26]:
shutdown_flag = np.diff(testdf['sum_eng_st'])
# shutdown_flag = np.roll(shutdown_flag,shift=-1)
# shutdown_flag[-1] = 0

## pd.merge(left on index)

In [27]:
testdf.reset_index().iloc[1200:1210]['sum_eng_st']

1200    199.0
1201    199.0
1202    199.0
1203    199.0
1204    199.0
1205    199.0
1206    200.0
1207    200.0
1208    200.0
1209    200.0
Name: sum_eng_st, dtype: float64

In [28]:
np.append(shutdown_flag,[0])[1205]

1.0

In [30]:
testdf['shutdown_flag'] = np.append(shutdown_flag,[0])
testdf.tail()

C:\Users\d59628\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,timestamp,psn,id,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,g_c_dt2,lo_c_dt5,c_dt5_1,c_dt5_2,c_dt5_3,c_dt5_4,c_dt5_5,c_dt5_6,sc_c_pct_e1,pe_c_pos_e1,f_c_pos_e2,f_c_pos_e1,c_c_t5_1,c_c_t5_2,c_c_t5_3,t5_s1,pe_p1,f_cmd1,pe_t1,perf_pow,sum_eng_h,lo_p1,b_p1,g_t1,t5_6,g_t5,g_t4,v_d_1b,g_cur1,g_cur4,g_cur3,lo_t9,f_cmd2,lo_t6,t5_5,t5_2,t5_3,t5_1,ngp,nt5,pcd,v_acc1,sc_pct2,g_pow1,pe_for1,sc_pct1,pe_cmd1,pe_pos1,g_cur2,t1_1,sum_esn,t5_4,f_p2,t5_a,sum_eng_st,lo_dp1,v_d_3b,f_p7,f_p1,v_d_2b,lo_t5,g_t2,f_t1,sum_enr,g_t3,g_pct1,f_pos1,shutdown_flag
1272265,2017-12-03 05:29:59.999996,46,363427,0.210789,0.101794,-0.085117,0.231336,1.010000,0.184000,0.113997,0.565882,0.057652,-0.135332,0.007523,0.050486,-0.099668,0.119338,-0.056264,0.014650,0.02002,0.00588,0.821902,0.436576,0.162800,0.305604,0.540706,0.750368,0.482424,0.503228,0.516600,0.859183,0.118859,0.829714,0.845769,0.2056,0.2184,0.114285,0.333984,0.334297,0.334788,0.826315,0.387870,0.906667,0.801968,0.794835,0.823406,0.833432,0.999541,1.0,0.236557,0.253333,0.961909,0.9442,0.194112,0.088133,0.374219,0.381410,0.334353,-0.05650,39.0,0.831999,0.616988,0.821902,208.0,0.232052,0.315999,14.621661,0.616545,0.248571,0.800435,0.809714,0.6225,5679.890625,0.2308,0.23534,0.750000,0.0
1272266,2017-12-03 05:40:00.000001,46,363428,0.208461,0.100745,-0.079453,0.234559,1.015000,0.184000,0.105998,0.563529,0.061930,-0.158131,-0.011089,0.046168,-0.072543,0.133664,-0.043599,-0.062849,0.00580,0.01328,0.820717,0.500220,0.245696,0.350154,0.540706,0.746824,0.482424,0.497943,0.516633,0.858407,0.118554,0.829714,0.847450,0.2032,0.2176,0.111428,0.328214,0.328750,0.329252,0.826842,0.384577,0.906667,0.806209,0.789091,0.818499,0.833103,1.000115,1.0,0.234242,0.253333,0.962278,0.9434,0.071472,0.088133,0.359502,0.367949,0.328728,-0.05875,39.0,0.829951,0.618169,0.820717,208.0,0.229488,0.319999,14.650033,0.617680,0.254285,0.799565,0.807428,0.6230,5679.973633,0.2300,0.26931,0.745994,0.0
1272267,2017-12-03 05:49:59.999995,46,363429,0.207771,0.083109,-0.068100,0.233561,1.018332,0.186001,0.118001,0.560001,0.052643,-0.134602,0.012543,0.039748,-0.087519,0.117187,-0.018000,-0.032900,-0.04560,-0.03376,0.821461,0.431637,0.165627,0.302146,0.540706,0.745486,0.481212,0.499715,0.516633,0.853755,0.121462,0.828572,0.844899,0.2052,0.2208,0.111428,0.328694,0.328683,0.329375,0.826842,0.382968,0.904667,0.803958,0.794541,0.823970,0.831990,1.000115,1.0,0.234968,0.253333,0.961909,0.9445,0.128325,0.088133,0.362917,0.370833,0.328895,-0.06125,39.0,0.829411,0.617579,0.821461,208.0,0.229488,0.323999,14.641521,0.617202,0.251428,0.796957,0.809143,0.6220,5680.057129,0.2284,0.25571,0.747596,0.0
1272268,2017-12-03 06:00:00.000000,46,363430,0.210325,0.074557,-0.096466,0.233969,1.011666,0.181998,0.115999,0.556469,0.055025,-0.147979,-0.013719,0.049292,-0.083763,0.141147,-0.033330,-0.007650,-0.00302,0.00478,0.823720,0.495643,0.235986,0.346950,0.540706,0.747895,0.476969,0.498671,0.516633,0.853755,0.124399,0.828000,0.851949,0.1972,0.2056,0.111428,0.331004,0.331641,0.331563,0.824737,0.385629,0.905334,0.806968,0.794124,0.820976,0.834725,0.999885,1.0,0.234196,0.248000,0.961909,0.9369,0.255838,0.088133,0.365612,0.373077,0.331384,-0.06175,39.0,0.833578,0.616988,0.823720,208.0,0.232052,0.315999,14.630173,0.617023,0.245714,0.796087,0.809143,0.6220,5680.140137,0.2236,0.24658,0.747596,0.0
1272269,2017-12-03 06:10:00.000004,46,363431,0.207542,0.088174,-0.079441,0.232426,1.013888,0.184000,0.119999,0.561176,0.049552,-0.129150,-0.004863,0.030928,-0.072056,0.125590,-0.058929,-0.048651,-0.01702,-0.00580,0.822506,0.436696,0.206098,0.305687,0.540706,0.748435,0.480000,0.500886,0.516633,0.855305,0.129374,0.827429,0.847624,0.2028,0.2096,0.111428,0.331964,0.331875,0.332377,0.825790,0.385955,0.904001,0.808095,0.796676,0.821533,0.832416,0.999885,1.0,0.235604,0.250667,0.962278,0.9438,0.320000,0.088133,0.371407,0.379808,0.332054,-0.06025,39.0,0.828691,0.616988,0.822506,208.0,0.229488,0.327999,14.625917,0.616664,0.245714,0.796957,0.809143,0.6220,5680.223145,0.2248,0.24391,0.748798